# pyds9 - Python connection to SAOimage DS9 via XPA

pyds9 on GitHub:  
https://github.com/ericmandel/pyds9

pyds9 documentation:  
http://hea-www.harvard.edu/RD/pyds9/index.html#indices-and-tables

XPA commands:  
http://ds9.si.edu/doc/ref/xpa.html


### Connect to ds9

For ds9 version 5.7 and beyond, simply register the
existing ds9 with the xpans name server by selecting the
ds9 File->XPA->Connect menu option.

In [1]:
import pyds9
d = pyds9.DS9()  # will open a new ds9 window or connect to an existing one


An instance of ds9 was found to be running before we could
start the 'xpans' name server. You will need to perform a
bit of manual intervention in order to connect this
existing ds9 to Python.

For ds9 version 5.7 and beyond, simply register the
existing ds9 with the xpans name server by selecting the
ds9 File->XPA->Connect menu option. Your ds9 will now be
fully accessible to pyds9 (e.g., it appear in the list
returned by the ds9_targets() routine).

For ds9 versions prior to 5.7, you cannot (easily) register
with xpans, but you can view ds9's File->XPA Information
menu option and pass the value associated with XPA_METHOD
directly to the Python DS9() constructor, e.g.:

    d = DS9('a000101:12345')

The good news is that new instances of ds9 will be
registered with xpans, and will be known to ds9_targets()
and the DS9() constructor.



### Example: Make a 2D gaussian image, view and manipulate it in ds9

from: http://www.astrobetter.com/blog/2010/07/14/manipulating-and-viewing-fits-files-in-python-with-pyds9/

In [2]:
from scipy import stats
import numpy as np
 
# Make a 2D gaussian image that is stored in a 2D numpy array
x = np.arange(-3, 3, 0.1)
xx, yy = np.meshgrid(x, x)
gauss2d = stats.norm.pdf(xx) * stats.norm.pdf(yy)

In [3]:
d = pyds9.DS9() 

# Load up our 2D gaussian
d.set_np2arr(gauss2d)

1

In [4]:
# Zoom to fit
d.set('zoom to fit')

1

In [5]:
# Change the colormap and scaling
d.set('cmap bb')
d.set('scale log')

1

In [6]:
# Add a label
d.set('regions command {text 30 20 #text="Fun with pyds9" font="times 18 bold"}')

1

In [7]:
# Check back to see what the current color scale is.
print(d.get('scale'))

log


### Example: lock and match wcs coordinates

In [8]:
from astropy.utils.data import get_pkg_data_filename

import pyds9
d = pyds9.DS9() 

d.set('frame delete all')

d.set('frame new')
# open first image in ds9, change scale
d.set('fits {}'.format(
    get_pkg_data_filename('galactic_center/gc_2mass_k.fits')))
d.set('scale asinh')
d.set('scale mode 99.5')
d.set('cmap a')

1

In [9]:
# open second image in ds9
d.set('frame new')

# open second image, change scale
d.set('fits {}'.format(
    get_pkg_data_filename('galactic_center/gc_msx_e.fits')))
d.set('scale asinh')
d.set('scale mode 99.5')
d.set('cmap heat')

1

In [10]:
# define function for matching and locking the wcs
def match_lock_wcs():
    d = pyds9.DS9()
    d.set('frame match wcs')
    d.set('frame lock wcs')

In [11]:
match_lock_wcs()

### Example: Make contours with different properties

In [12]:
from astropy.utils.data import get_pkg_data_filename

d = pyds9.DS9() 

d.set('frame delete all')

d.set('frame new')
# open first image in ds9, change scale
d.set('fits {}'.format(
    get_pkg_data_filename('galactic_center/gc_bolocam_gps.fits')))

d.set('scale linear')
d.set('scale mode 99.5')
d.set('cmap grey')


1

In [13]:
d.set('contour yes')

for value, color, width, dash in zip(
        [1, 3, 5, 7, 9],
        ['green', 'black', 'blue', 'red', 'yellow'],
        [1, 2, 2, 3, 3],
        ['yes', 'no', 'yes', 'no', 'no']):
    d.set('contour color {}'.format(color))
    d.set('contour limits {} {}'.format(value, value))
    d.set('contour scale linear')
    d.set('contour nlevels 1')
    d.set('contour smooth 1')
    d.set('contour width {}'.format(width))
    d.set('contour dash {}'.format(dash))
    d.set('contour generate')
    d.set('contour convert')
    
d.set('contour no')
d.set('mode pan')


1

### Example: Make an RGB image in ds9

In [14]:
from astropy.utils.data import get_pkg_data_filename

d = pyds9.DS9() 

d.set('frame delete all')

imgList = ['galactic_center/gc_2mass_h.fits',
           'galactic_center/gc_2mass_j.fits',
           'galactic_center/gc_2mass_k.fits']

for img in imgList:
    d.set('frame new')
    d.set('scale linear')
    d.set('scale mode 99.5')
    d.set('fits {}'.format(get_pkg_data_filename(img)))
    
match_lock_wcs()


In [15]:
d = pyds9.DS9()
d.set('frame new rgb')
d.set('rgb red')  # set current channel to red
d.set('rgb open')
d.set('rgb lock wcs yes')
d.set('rgb lock crop yes')
d.set('rgb lock slice yes')
d.set('rgb lock bin yes')
d.set('rgb lock scale no')
d.set('rgb lock colorbar no')
d.set('rgb lock smooth yes')
d.set('rgb close')
d.set('fits slice {}'.format(get_pkg_data_filename(imgList[2])))
d.set('scale asinh')
d.set('scale mode 99.5')
d.set('rgb open')
d.set('rgb green')  # set current channel to green
d.set('rgb close')
d.set('fits slice {}'.format(get_pkg_data_filename(imgList[0])))
d.set('scale asinh')
d.set('scale mode 99.5')
d.set('rgb open')
d.set('rgb blue')  # set current channel to blue
d.set('rgb close')
d.set('fits slice {}'.format(get_pkg_data_filename(imgList[1])))
d.set('scale asinh')
d.set('scale mode 99.5')


1

### Example: Let's go interactive!

In [16]:
from astropy.utils.data import get_pkg_data_filename

import pyds9
d = pyds9.DS9() 

d.set('frame delete all')

d.set('frame new')
# open first image in ds9, change scale
d.set('fits {}'.format(
    get_pkg_data_filename('galactic_center/gc_2mass_k.fits')))
d.set('scale asinh')
d.set('scale mode 99.5')
d.set('cmap grey')


1

In [17]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def ds9_slide(txt, zoom=0.4, scale=99.5, mode='asinh'):
    d = pyds9.DS9()
    d.set('regions delete all')
    d.set('scale {}'.format(mode))
    d.set('scale mode {}'.format(scale))
    d.set('zoom to {}'.format(zoom))
    d.set('regions command {{text 300 30 #text="{}" color=red font="times 18 bold"}}'.format(txt))
    #return scale, mode, zoom

lstScale = ['linear', 'log', 'asinh']
interact(ds9_slide, 
         scale=(50, 100, 0.01),
         zoom=(0.1, 50, 0.1),
         mode=lstScale,
         txt='Fun with pyds9')


In [ ]:
d.set('regions delete all')

In [18]:
d.get('regions')

'# Region file format: DS9 version 4.1\nglobal color=green dashlist=8 3 width=1 font="helvetica 10 normal roman" select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nfk5\n# text(17:45:59.337,-29:23:32.326) color=red font="times 18 bold" text={sdfjkjxyv sjdf}\ncircle(17:44:44.370,-28:54:55.240,308.141") # color=red font="times 18 bold "\n'